In [1]:
import requests
import pandas as pd
import os
from env import host, user, password

In [2]:
# def read_csv(url):
#     '''
#     This function reads in the csv from the url
#     '''
#     df = pd.read_csv(url)
#     return df

In [10]:
# Trying to read in csv from folder
os.chdir("fitbit")
df = pd.read_csv("2018-04-26_through_2018-05-26.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'fitbit'

In [ ]:
r'Path where the CSV file is stored\File name.csv')